In [1]:
# !pip install gekko
from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import json

In [2]:
def print_results(history):
    """
    A function for results printing
    """
    name = history['name']
    print(f'Results for {name}')
    check = lambda x: 'x' in x and x != 'x_final'
    x_keys = [(item[0], float(item[1][-1])) for item in history.items() if check(item[0])]
    for key, x in x_keys:
        print(f'{key}: {x:.4f}')
    obj = history['obj_final']
    print(f'Objective {obj:.4f}')

**Optimal Control** problem:

* $x$ - state value
* $u$ - control variable
* $t$ - time variable $\in [t_0, t_f] = N$ - time horizon

**Continuous time**: \
Cost function: $F(\cdot) = V_f(x(t_0), t_0, x(t_f), t_f) + \int\limits_{t_0}^{t_f} l(x(t), u(t), t) dt$
* State equation: $\dot{x} = f(x, u)$ if time invariant else $\dot{x} = f(x, u, t)$
* Path constraints: $h(x(t), u(t), t) \leq 0$
* Final constraint: $h_f(x(t_f), u(t_f), t_f) \leq 0$


**Discrete time**: \
Cost function: $F(\cdot) = V_f(x(t_0), t_0, x(t_f), t_f) + \sum\limits_{t=t_0}^{t_f} l(x(t), u(t), t)$
* State equation: $x^{+} = f(x, u)$ if time invariant else $x^{+} = f(x, u, t)$
* Path constraints: $h(x(t), u(t), t) \leq 0$
* Final constraint: $h_f(x(t_f), u(t_f), t_f) \leq 0$

Moreover, in this case we will use the notation:
$t \in [t_0, t_f] \mapsto k \in [0, N]: x(t) = x(k)$

Now suppose we have

* $u \in \mathbb{R}^{n_u}$ - continuous control variable
* $i \in \mathbb{Z}^{n_i}$ - integer control variable inside a bounded convex polyhedron P
* $F$ consists of a nonlinear least squares term $F1$ and nonlinear term $F_2$ - both are differentiable

And lets note
* $\textbf{x} = (x(0), x(1), \dots, x(N))$
* $\textbf{u} = (u(0), u(1), \dots, u(N-1))$
* $\textbf{i} = (i(0), i(1), \dots, i(N-1))$

So we have **Mixed-Integer Optimal Control** problem:

$
\min\limits_{\textbf{x}, \textbf{u}, \textbf{i}}
{F(\textbf{x}, \textbf{u}, \textbf{i})}, \quad
\left\{ \begin{array}{l}
x(0) = x_0 \\
x(k+1) = f(x(k), u(k), i(k)) \\
h(x(k), u(k), i(k)) \leq 0, k < N \\
h_f(x(N)) \leq 0 \\
\textbf{i} \in P \\
\textbf{i} \in \mathbb{Z}^{N \cdot n_i}
\end{array} \right.
\;\; $ and 
$
\;\; 
\left\{ \begin{array}{l}
F(\textbf{x}, \textbf{u}, \textbf{i}) = 
V_f(x(N)) + \sum\limits_{k=0}^{N} l(x(k), u(k), i(k)) \\
\left. \begin{array}{l}
l(x, u, i) = 
\frac{1}{2} \|l_1(x, u, i)\|_2^2 + l_2(x, u, i) \\
V_f(x) = 
\frac{1}{2} \|V_1(x)\|_2^2 + V_2(x)
\end{array} \right\}
\Rightarrow
F(\textbf{x}, \textbf{u}, \textbf{i}) = 
\frac{1}{2} \|F_1(x, u, i)\|_2^2 + F_2(x, u, i) \\
\end{array} \right.
$

**Notational simplifications**:

* $z = (\textbf{x}, \textbf{u}) \in \mathbb{R}^{n_z}$, where $n_z = (N+1) \cdot n_x + N \cdot n_u$ - continuous variable
* $y = \textbf{i} \in \mathbb{R}^{n_y}$, where $n_y = N \cdot n_i$ - integer variable

Then 
* $F(\textbf{x}, \textbf{u}, \textbf{i}) = F(y, z)$
* $x(0) = x_0, \; x(k+1) = f(\dots) \rightarrow G(y, z)$
* $h, h_f \rightarrow H(y, z)$
* $i \in P \rightarrow y \in P$
* $i \in \mathbb{Z}^{N \cdot n_i} \rightarrow y \in \mathbb{Z}^{n_y}$

Now our **MI Optimal Control** problem just is a **MINL Programming** problem.

**Example**

$
\begin{array}{rll}
\text{continuous time}: & 
    \dot{x}(t) = f_c(x(t), u(t)) = x^3(t) - u(t) \\
\text{discrete time}: & 
    x^{+} = f(x, u) = \text{Runge-Kutta-4}\ (f_c)
\end{array}
$

$
\text{objetive function}:
F(\textbf{x}, \textbf{u}) = \frac{1}{2} \sum\limits^{N}_{k=0} (x(k) - x_{ref})^2
$

$
P = 
\left\{ \textbf{u} \in [0, 1]^N \mid \begin{align} 
    & u(k) \geq u(k-1) - u(k-2) \\
    & u(k) \geq u(k-1) - u(k-3) 
\end{align} \right\}
$

$
\textbf{MINLP}: 
\min\limits_{\textbf{x}, \textbf{u}}
F(\textbf{x}, \textbf{u}), \quad
\text{such that} \;\;
\left\{ \begin{array}{l}
x(0) = x_0 \\
x(k+1) = f(x(k), u(k)) : \textbf{G}(\textbf{x}, \textbf{u}) \\
\textbf{u} \in P \\
\textbf{u} \in \mathbb{Z}^{N} \\
\end{array} \right.
$

$N = 30, \ x_0 = 0.8, \ x_{ref} = 0.7$

In [45]:
# ODE
def f(x, u):
    return x ** 3 - u

# Constraints: equality 
def G(x, u):
    return x[1:] == f(x[:-1], u)

# Constraints: inequality
def H(x, u):
    pass

# Convex polyhedron
def P(u):
    def check(k):
        a = u[k-1] if k > 0 else 0
        b = u[k-2] if k > 1 else 0
        c = u[k-3] if k > 2 else 0
        flag_0 = u[k] >= a - b
        flag_1 = u[k] >= a - c
        return flag_0 & flag_1
    return all([check(k) for k, _ in enumerate(u)])

# Nonlinear least squares term F1
def F1(x, u, x_ref):
    return ((x - x_ref) ** 2).sum() / 2

# Nonlinear term F2
def F2(x, u):
    return 0

# Objective function
def F(x, u, x_ref):
    return F1(x, u, x_ref) + F2(x, u)

In [ ]:
def run(x_0, x_ref, G, H, F, integer=False, const=False, name=''):
    
    history = {'x' : [] , 'u' : []}
    history['obj'] = []

    max_iter = 0
    while True:
        m = GEKKO(remote=False)                  # Initialize gekko
        m.options.SOLVER = 1 if integer else 3   # Define the solver

        x = m.Var(value=x_0, lb=1, ub=5)

        # Integer variables
        u = m.Const(value=u_0) if const else m.Var(value=u_0, lb=0, ub=1, integer=integer)

        # Equations
        m.Equation(G(x, u)) # G function

        # Objective
        m.Obj(F(x, u))

        m.options.MAX_ITER = max_iter
        m.solve(disp=False,debug=0)

        history['x'].append(np.array(x.value))
        history['xu'].append(np.array(u.value))
        history['obj'].append(m.options.OBJFCNVAL)

        if m.options.APPSTATUS==1:
            break
        else:
            max_iter += 1

#     x = []
#     for key in history.keys():
#         history[key] = np.ravel(history[key])
#         x.append(history[key][-1])
#     history['x_final'] = x[:-1]
#     history['obj_final'] = x[-1]
#     history['name'] = name

    return history